In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets torch python-dotenv
!pip install huggingface_hub

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [6]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

In [7]:
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sent_train.csv:   0%|          | 0.00/859k [00:00<?, ?B/s]

sent_valid.csv:   0%|          | 0.00/217k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [8]:
def tokenize_function(examples):
    # Tokenize the input
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",  # Ensure padding to max length
        truncation=True,
        max_length=512,  # Set an appropriate max length based on your use case
    )

    # Shift labels to create the next-token prediction
    tokenized["labels"] = tokenized["input_ids"].copy()
    # Optionally shift labels for the prediction task
    tokenized["labels"] = [ids[1:] + [tokenizer.pad_token_id] for ids in tokenized["labels"]]

    return tokenized

# Map the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/9543 [00:00<?, ? examples/s]

Map:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [9]:
!pip install accelerate peft


In [10]:
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset


In [11]:
lora_config = LoraConfig(
    r=16,                # Low-rank adaptation rank (adjust as needed)
    lora_alpha=32,      # Alpha for the LoRA updates
    lora_dropout=0.1,   # Dropout rate
    bias="none",        # Bias setting
    task_type=TaskType.CAUSAL_LM,  # Task type for causal language modeling
)

# Wrap the model with QLoRA
model = get_peft_model(model, lora_config)


In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_steps=10_000,
    per_device_train_batch_size=1,  # Adjust as necessary
    per_device_eval_batch_size=1,   # Adjust as necessary
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=200,
    fp16=True,                      # Enable mixed precision
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss
200,0.929100,0.328822
400,0.314000,0.302559
600,0.304300,0.290621
800,0.301000,0.284656
1000,0.287600,0.281897
1200,0.261400,0.279227
1400,0.270800,0.277067
1600,0.259300,0.275129
1800,0.279100,0.273888
2000,0.283700,0.273762


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [ ]:
trainer.save_model("/content/drive/MyDrive/Projects/Financial Sentiment Prediction using LLM")

In [ ]:
model_name = "/content/drive/MyDrive/Projects/Financial Sentiment Prediction using LLM"  # Replace with your model path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
prompt = "Analysts hoping a high return on Nvidia"
inputs = tokenizer(prompt, return_tensors="pt")

In [ ]:
# Generate output
outputs = model.generate(
    inputs["input_ids"],
    max_length=100,
    num_return_sequences=1,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    do_sample=True
)

In [ ]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)